Arbol elemental con libreria  rpart

#Debe tener instaladas las librerias  data.table  ,  rpart   y rpart.plot

In [ ]:
#cargo las librerias que necesito
require("data.table")
require("rpart")
require("rpart.plot")

In [ ]:
#Aqui se debe poner la carpeta de SU computadora local
setwd("D:\\gdrive\\Austral2022R\\")  #Establezco el Working Directory

cargo los datos de 202009 que es donde voy a ENTRENAR el modelo

In [ ]:
dtrain  <- fread("./datasets/paquete_premium_202011.csv")

genero el modelo,  aqui se construye el arbol

In [ ]:
modelo  <- rpart("clase_ternaria ~ .",
                 data = dtrain,
                 xval=0,
                 cp=        -0.3,  #esto significa no limitar la complejidad de los splits
                 minsplit=  100,   #minima cantidad de registros para que se haga el split
                 minbucket=   1,   #tamaño minimo de una hoja
                 maxdepth=   10 )  #PRUEBO  una profundidad de 10

Ahora grafico el arbol, va a ser muy frondoso porque la profundidad es de 10

In [ ]:
prp(modelo, extra=101, digits=5, branch=1, type=4, varlen=0, faclen=0)

In [ ]:
#Ahora aplico al modelo  a los datos de 202101  y genero la salida para kaggle

#cargo los datos de 202011, que es donde voy a APLICAR el modelo

In [ ]:
dapply  <- fread("./datasets/paquete_premium_202101.csv")

In [ ]:
prediccion  <- predict( modelo, dapply , type = "prob") #aplico el modelo

prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
cada columna es el vector de probabilidades 

agrego a dapply una columna nueva que es la probabilidad de BAJA+2

In [ ]:
dapply[ , prob_baja2 := prediccion[, "BAJA+2"] ]

solo le envio estimulo a los registros con probabilidad de BAJA+2 mayor  a  1/60

In [ ]:
dapply[ , Predicted  := as.numeric(prob_baja2 > 1/60) ]

genero un dataset con las dos columnas que me interesan

In [ ]:
entrega  <- dapply[   , list(numero_de_cliente, Predicted) ] #genero la salida

#Finalmente genero el archivo para Kaggle

In [ ]:
#creo la carpeta donde va el experimento

In [ ]:
dir.create( "./labo/exp/" ) 
dir.create( "./labo/exp/KA2002" ) 

grabo el dataset

In [ ]:
fwrite( entrega, 
        file= "./labo/exp/KA2002/K102_001.csv", 
        sep= "," )